In [52]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [53]:
import warnings

warnings.filterwarnings( 'ignore' )

In [54]:
train_data = pd.read_csv("train.csv") 
test_data = pd.read_csv("test.csv")

In [55]:
x_data = train_data.loc[:, 'f0':'f99']
y_data = train_data.loc[:, 'loss']

In [56]:
from sklearn.model_selection import train_test_split

In [57]:
x_train, x_test, y_train, y_test=train_test_split(x_data,
                                                  y_data,
                                                  test_size=0.2,   #전체 중 20%를 테스트용으로 분할
                                                                   #나머지 80%는 훈련용
                                                  shuffle=True,    #무작위로 섞어서 추출
                                                  random_state=20) #무작위 추출 시 일정한 기준으로

In [58]:
# catboost 참고 코드: https://catboost.ai/docs/concepts/python-usages-examples.html
# parameter tuning: https://catboost.ai/docs/concepts/parameter-tuning.html
from catboost import CatBoostRegressor
import catboost as cb

In [59]:
train_pool = cb.Pool(x_train, y_train)
test_pool = cb.Pool(x_test)

In [91]:
model = CatBoostRegressor(iterations=10000,
                          learning_rate=0.01,
                          depth=8,
                          random_seed = 20,
                          l2_leaf_reg = 1,
                          loss_function='RMSE')
model.fit(train_pool)

0:	learn: 7.9500375	total: 47.5ms	remaining: 7m 55s
1:	learn: 7.9496164	total: 77.1ms	remaining: 6m 25s
2:	learn: 7.9492057	total: 106ms	remaining: 5m 53s
3:	learn: 7.9487596	total: 134ms	remaining: 5m 35s
4:	learn: 7.9483366	total: 163ms	remaining: 5m 26s
5:	learn: 7.9478263	total: 192ms	remaining: 5m 19s
6:	learn: 7.9474155	total: 220ms	remaining: 5m 14s
7:	learn: 7.9469625	total: 249ms	remaining: 5m 11s
8:	learn: 7.9465283	total: 282ms	remaining: 5m 12s
9:	learn: 7.9460816	total: 312ms	remaining: 5m 11s
10:	learn: 7.9456236	total: 341ms	remaining: 5m 9s
11:	learn: 7.9452673	total: 369ms	remaining: 5m 6s
12:	learn: 7.9448423	total: 395ms	remaining: 5m 3s
13:	learn: 7.9445072	total: 424ms	remaining: 5m 2s
14:	learn: 7.9441052	total: 451ms	remaining: 5m
15:	learn: 7.9437393	total: 482ms	remaining: 5m
16:	learn: 7.9433697	total: 512ms	remaining: 5m
17:	learn: 7.9430102	total: 541ms	remaining: 4m 59s
18:	learn: 7.9425945	total: 570ms	remaining: 4m 59s
19:	learn: 7.9422524	total: 597ms	re

163:	learn: 7.9054314	total: 4.76s	remaining: 4m 45s
164:	learn: 7.9052533	total: 4.79s	remaining: 4m 45s
165:	learn: 7.9050768	total: 4.82s	remaining: 4m 45s
166:	learn: 7.9048474	total: 4.85s	remaining: 4m 45s
167:	learn: 7.9046383	total: 4.88s	remaining: 4m 45s
168:	learn: 7.9044396	total: 4.91s	remaining: 4m 45s
169:	learn: 7.9042064	total: 4.93s	remaining: 4m 45s
170:	learn: 7.9040015	total: 4.96s	remaining: 4m 45s
171:	learn: 7.9037893	total: 4.99s	remaining: 4m 45s
172:	learn: 7.9035859	total: 5.02s	remaining: 4m 45s
173:	learn: 7.9033841	total: 5.04s	remaining: 4m 44s
174:	learn: 7.9031887	total: 5.07s	remaining: 4m 44s
175:	learn: 7.9030108	total: 5.1s	remaining: 4m 44s
176:	learn: 7.9028343	total: 5.13s	remaining: 4m 44s
177:	learn: 7.9026232	total: 5.16s	remaining: 4m 44s
178:	learn: 7.9024375	total: 5.18s	remaining: 4m 44s
179:	learn: 7.9022311	total: 5.22s	remaining: 4m 44s
180:	learn: 7.9020651	total: 5.24s	remaining: 4m 44s
181:	learn: 7.9018984	total: 5.27s	remaining: 4

323:	learn: 7.8764048	total: 9.38s	remaining: 4m 40s
324:	learn: 7.8762726	total: 9.41s	remaining: 4m 40s
325:	learn: 7.8761179	total: 9.44s	remaining: 4m 40s
326:	learn: 7.8759770	total: 9.47s	remaining: 4m 40s
327:	learn: 7.8758005	total: 9.5s	remaining: 4m 40s
328:	learn: 7.8756203	total: 9.53s	remaining: 4m 40s
329:	learn: 7.8754386	total: 9.56s	remaining: 4m 40s
330:	learn: 7.8752948	total: 9.59s	remaining: 4m 40s
331:	learn: 7.8751308	total: 9.62s	remaining: 4m 40s
332:	learn: 7.8749560	total: 9.65s	remaining: 4m 40s
333:	learn: 7.8747914	total: 9.68s	remaining: 4m 40s
334:	learn: 7.8746449	total: 9.71s	remaining: 4m 40s
335:	learn: 7.8745041	total: 9.73s	remaining: 4m 39s
336:	learn: 7.8743849	total: 9.76s	remaining: 4m 39s
337:	learn: 7.8742062	total: 9.79s	remaining: 4m 39s
338:	learn: 7.8740368	total: 9.83s	remaining: 4m 40s
339:	learn: 7.8738629	total: 9.86s	remaining: 4m 40s
340:	learn: 7.8736830	total: 9.88s	remaining: 4m 39s
341:	learn: 7.8734754	total: 9.91s	remaining: 4

485:	learn: 7.8508589	total: 14.2s	remaining: 4m 37s
486:	learn: 7.8506837	total: 14.2s	remaining: 4m 37s
487:	learn: 7.8505287	total: 14.2s	remaining: 4m 37s
488:	learn: 7.8503757	total: 14.3s	remaining: 4m 37s
489:	learn: 7.8502257	total: 14.3s	remaining: 4m 37s
490:	learn: 7.8500706	total: 14.3s	remaining: 4m 37s
491:	learn: 7.8499311	total: 14.3s	remaining: 4m 37s
492:	learn: 7.8497791	total: 14.4s	remaining: 4m 37s
493:	learn: 7.8496461	total: 14.4s	remaining: 4m 37s
494:	learn: 7.8494806	total: 14.4s	remaining: 4m 37s
495:	learn: 7.8493182	total: 14.5s	remaining: 4m 37s
496:	learn: 7.8492046	total: 14.5s	remaining: 4m 37s
497:	learn: 7.8490749	total: 14.5s	remaining: 4m 37s
498:	learn: 7.8489142	total: 14.6s	remaining: 4m 37s
499:	learn: 7.8487847	total: 14.6s	remaining: 4m 37s
500:	learn: 7.8486230	total: 14.6s	remaining: 4m 36s
501:	learn: 7.8484968	total: 14.6s	remaining: 4m 36s
502:	learn: 7.8483520	total: 14.7s	remaining: 4m 36s
503:	learn: 7.8482048	total: 14.7s	remaining: 

647:	learn: 7.8273869	total: 19.2s	remaining: 4m 36s
648:	learn: 7.8272682	total: 19.2s	remaining: 4m 36s
649:	learn: 7.8271432	total: 19.2s	remaining: 4m 36s
650:	learn: 7.8269815	total: 19.3s	remaining: 4m 36s
651:	learn: 7.8268647	total: 19.3s	remaining: 4m 36s
652:	learn: 7.8266976	total: 19.3s	remaining: 4m 36s
653:	learn: 7.8265460	total: 19.4s	remaining: 4m 36s
654:	learn: 7.8264036	total: 19.4s	remaining: 4m 36s
655:	learn: 7.8262350	total: 19.4s	remaining: 4m 36s
656:	learn: 7.8260916	total: 19.5s	remaining: 4m 36s
657:	learn: 7.8259492	total: 19.5s	remaining: 4m 36s
658:	learn: 7.8258024	total: 19.5s	remaining: 4m 36s
659:	learn: 7.8256733	total: 19.5s	remaining: 4m 36s
660:	learn: 7.8255186	total: 19.6s	remaining: 4m 36s
661:	learn: 7.8253706	total: 19.6s	remaining: 4m 37s
662:	learn: 7.8252370	total: 19.7s	remaining: 4m 37s
663:	learn: 7.8250676	total: 19.7s	remaining: 4m 37s
664:	learn: 7.8249283	total: 19.8s	remaining: 4m 37s
665:	learn: 7.8247872	total: 19.8s	remaining: 

...

9310:	learn: 6.7873271	total: 4m 47s	remaining: 21.3s
9311:	learn: 6.7871959	total: 4m 47s	remaining: 21.2s
9312:	learn: 6.7870629	total: 4m 47s	remaining: 21.2s
9313:	learn: 6.7869421	total: 4m 47s	remaining: 21.2s
9314:	learn: 6.7868654	total: 4m 47s	remaining: 21.1s
9315:	learn: 6.7867457	total: 4m 47s	remaining: 21.1s
9316:	learn: 6.7866260	total: 4m 47s	remaining: 21.1s
9317:	learn: 6.7865252	total: 4m 47s	remaining: 21.1s
9318:	learn: 6.7864185	total: 4m 47s	remaining: 21s
9319:	learn: 6.7863250	total: 4m 47s	remaining: 21s
9320:	learn: 6.7862010	total: 4m 47s	remaining: 21s
9321:	learn: 6.7861147	total: 4m 47s	remaining: 20.9s
9322:	learn: 6.7859777	total: 4m 47s	remaining: 20.9s
9323:	learn: 6.7858690	total: 4m 47s	remaining: 20.9s
9324:	learn: 6.7857341	total: 4m 47s	remaining: 20.8s
9325:	learn: 6.7856376	total: 4m 47s	remaining: 20.8s
9326:	learn: 6.7855219	total: 4m 48s	remaining: 20.8s
9327:	learn: 6.7854135	total: 4m 48s	remaining: 20.8s
9328:	learn: 6.7852939	total: 4m 4

9468:	learn: 6.7700549	total: 4m 52s	remaining: 16.4s
9469:	learn: 6.7699451	total: 4m 52s	remaining: 16.4s
9470:	learn: 6.7698263	total: 4m 52s	remaining: 16.3s
9471:	learn: 6.7697345	total: 4m 52s	remaining: 16.3s
9472:	learn: 6.7696261	total: 4m 52s	remaining: 16.3s
9473:	learn: 6.7695053	total: 4m 52s	remaining: 16.3s
9474:	learn: 6.7693921	total: 4m 52s	remaining: 16.2s
9475:	learn: 6.7692711	total: 4m 52s	remaining: 16.2s
9476:	learn: 6.7691507	total: 4m 52s	remaining: 16.2s
9477:	learn: 6.7690526	total: 4m 52s	remaining: 16.1s
9478:	learn: 6.7689628	total: 4m 52s	remaining: 16.1s
9479:	learn: 6.7688623	total: 4m 52s	remaining: 16.1s
9480:	learn: 6.7687296	total: 4m 53s	remaining: 16s
9481:	learn: 6.7686171	total: 4m 53s	remaining: 16s
9482:	learn: 6.7685007	total: 4m 53s	remaining: 16s
9483:	learn: 6.7683856	total: 4m 53s	remaining: 15.9s
9484:	learn: 6.7683040	total: 4m 53s	remaining: 15.9s
9485:	learn: 6.7681673	total: 4m 53s	remaining: 15.9s
9486:	learn: 6.7680447	total: 4m 5

9623:	learn: 6.7532457	total: 4m 57s	remaining: 11.6s
9624:	learn: 6.7531176	total: 4m 57s	remaining: 11.6s
9625:	learn: 6.7529807	total: 4m 57s	remaining: 11.6s
9626:	learn: 6.7528898	total: 4m 57s	remaining: 11.5s
9627:	learn: 6.7527714	total: 4m 57s	remaining: 11.5s
9628:	learn: 6.7526691	total: 4m 57s	remaining: 11.5s
9629:	learn: 6.7525367	total: 4m 57s	remaining: 11.4s
9630:	learn: 6.7524224	total: 4m 57s	remaining: 11.4s
9631:	learn: 6.7522966	total: 4m 58s	remaining: 11.4s
9632:	learn: 6.7521976	total: 4m 58s	remaining: 11.4s
9633:	learn: 6.7520621	total: 4m 58s	remaining: 11.3s
9634:	learn: 6.7519618	total: 4m 58s	remaining: 11.3s
9635:	learn: 6.7518263	total: 4m 58s	remaining: 11.3s
9636:	learn: 6.7517077	total: 4m 58s	remaining: 11.2s
9637:	learn: 6.7515979	total: 4m 58s	remaining: 11.2s
9638:	learn: 6.7514950	total: 4m 58s	remaining: 11.2s
9639:	learn: 6.7513796	total: 4m 58s	remaining: 11.1s
9640:	learn: 6.7512759	total: 4m 58s	remaining: 11.1s
9641:	learn: 6.7511594	total

9782:	learn: 6.7358341	total: 5m 2s	remaining: 6.72s
9783:	learn: 6.7357037	total: 5m 2s	remaining: 6.69s
9784:	learn: 6.7355761	total: 5m 2s	remaining: 6.66s
9785:	learn: 6.7354818	total: 5m 2s	remaining: 6.63s
9786:	learn: 6.7353778	total: 5m 3s	remaining: 6.59s
9787:	learn: 6.7352711	total: 5m 3s	remaining: 6.56s
9788:	learn: 6.7351593	total: 5m 3s	remaining: 6.53s
9789:	learn: 6.7350488	total: 5m 3s	remaining: 6.5s
9790:	learn: 6.7349486	total: 5m 3s	remaining: 6.47s
9791:	learn: 6.7348506	total: 5m 3s	remaining: 6.44s
9792:	learn: 6.7347429	total: 5m 3s	remaining: 6.41s
9793:	learn: 6.7346207	total: 5m 3s	remaining: 6.38s
9794:	learn: 6.7344878	total: 5m 3s	remaining: 6.35s
9795:	learn: 6.7343604	total: 5m 3s	remaining: 6.32s
9796:	learn: 6.7342390	total: 5m 3s	remaining: 6.29s
9797:	learn: 6.7341415	total: 5m 3s	remaining: 6.25s
9798:	learn: 6.7340281	total: 5m 3s	remaining: 6.22s
9799:	learn: 6.7338779	total: 5m 3s	remaining: 6.19s
9800:	learn: 6.7337882	total: 5m 3s	remaining: 

9940:	learn: 6.7187348	total: 5m 8s	remaining: 1.83s
9941:	learn: 6.7186101	total: 5m 8s	remaining: 1.8s
9942:	learn: 6.7184938	total: 5m 8s	remaining: 1.77s
9943:	learn: 6.7183765	total: 5m 8s	remaining: 1.74s
9944:	learn: 6.7182440	total: 5m 8s	remaining: 1.71s
9945:	learn: 6.7181225	total: 5m 8s	remaining: 1.67s
9946:	learn: 6.7179737	total: 5m 8s	remaining: 1.64s
9947:	learn: 6.7178672	total: 5m 8s	remaining: 1.61s
9948:	learn: 6.7177209	total: 5m 8s	remaining: 1.58s
9949:	learn: 6.7175960	total: 5m 8s	remaining: 1.55s
9950:	learn: 6.7174929	total: 5m 8s	remaining: 1.52s
9951:	learn: 6.7174215	total: 5m 8s	remaining: 1.49s
9952:	learn: 6.7173177	total: 5m 8s	remaining: 1.46s
9953:	learn: 6.7172471	total: 5m 8s	remaining: 1.43s
9954:	learn: 6.7171252	total: 5m 8s	remaining: 1.4s
9955:	learn: 6.7170121	total: 5m 8s	remaining: 1.36s
9956:	learn: 6.7168917	total: 5m 8s	remaining: 1.33s
9957:	learn: 6.7167706	total: 5m 8s	remaining: 1.3s
9958:	learn: 6.7166541	total: 5m 8s	remaining: 1.

In [89]:
from sklearn.metrics import mean_squared_error
preds = model.predict(test_pool)
rms = np.sqrt(mean_squared_error(y_test, preds))
print(rms)

7.793327223673384


In [68]:
# depth = 6

# model = CatBoostRegressor(iterations=10000,
#                           learning_rate=0.03,
#                           depth=6,
#                           random_seed = 20,
#                           early_stopping_rounds = 5,
#                           loss_function='RMSE')
# rmse:7.813845889737223
# score:7.90029

# iter: 50000, learning_rate = 0.03
# rmse: 7.923826522022192

# iter: 5000, learning_rate = 0.03
# rmse: 7.799099906546834
# score: 7.89085

# iter: 3000, leaning_rate = 0.03 
# rmse: 7.796606801724997

# iter: 2000, learning_rate = 0.03
# rmse: 7.7980331449936235

# iter: 3000, learning_rate = 0.05
# rmse: 7.807504189243877

# iter: 3000, learning_rate = 0.01
# rmse: 7.807666005260566

# iter: 4000, learning_rate = 0.01
# rmse: 7.807666005260566

# iter: 7000, learning_rate = 0.01
# rmse: 7.7954222999385046

# iter: 10000, learning_rate = 0.01
# rmse: 7.793435521881254

# iter: 20000, learning_rate = 0.01
# rmse: 7.793909233479778

# iter: 10000, learning_rate = 0.01, L2 regularization (best)
# rmse: 7.793327223673384

In [90]:
new_x_test = test_data.loc[:, 'f0':'f99']
new_y_pred =  model.predict(new_x_test)


from pandas import Series, DataFrame

raw_data = {'id': [ i for i in range(250000,400000)],
            'loss': new_y_pred }

data = DataFrame(raw_data)
data.set_index('id', inplace=True)
print(data)
data.to_csv("submission.csv", mode='w')

            loss
id              
250000  8.134695
250001  4.437273
250002  8.520938
250003  7.183588
250004  7.617621
...          ...
399995  7.778029
399996  7.007706
399997  5.811241
399998  5.751366
399999  6.511879

[150000 rows x 1 columns]
